In [1]:
import requests
import json
from PIL import Image
from io import BytesIO
import re
import demjson
import pandas as pd
from pandas import DataFrame
from datetime import datetime, timedelta , date
import numpy as np
import ast
from pandas.io.json import json_normalize

In [2]:
# r = requests.post("http://192.168.1.30:88/SYSA/mobilephone/getyzm.asp?serialnum=616EA64CD6A65AA")
# print(r.status_code, r.reason)
# i = Image.open(BytesIO(r.content))

In [3]:
rowsPerPage = 100000

today = date.today()

yesterday = date.today() - timedelta(1)
yesterday = yesterday.strftime('%Y-%m-%d')

start_date = "2018-05-26"

start_date, end_date = yesterday, yesterday

In [4]:
rndcode = ""
user = "pig"
password = "chimo123"
serialnum = "616EA64CD6A65AA"

data ='{session:"******",datas:[{id:"user",val:"txt:%s"},{id:"password",val:"txt:%s"},{id:"serialnum",val:"%s"},{id:"rndcode",val:"%s"}]}'%(user,password,serialnum,rndcode)
        
headers = {"Content-Type":"application/zsml;charset=gb2312"}
res = requests.post("http://192.168.1.30:88/SYSA/mobilephone/login.asp",data=data,headers=headers)
# print(res.status_code)
ses_text = res.text
ses_text

'{header:{status:0,message:"OK",language:"cn",session:"e41a0c259979e294638da27162a3d0e81d2936a2b4485f6d18a926714f1e18b228b250cd00811a94ad5fa748627efba6ebfdc00c5582a0ee"},body:{action:"sys",model:"init",sys:{info:{floatbit:2,moneybit:2,disbit:2,dismax:1,hlbit:2,mxbit:500,remind:300,isopenrecord:false,ratebit:300,uniqueName:"cdb90870289093b61cc89ae72f786c74"},trees:{nodes:[{id:"SystemManage",action:"_none",text:"系统操作",nodes:[{id:"about",ico:"about",action:"_client",text:"关于",url:"100103",nodes:[],deep:0}],deep:0}],deep:0},homeurl:"mobilephone/home2.asp",exiturl:"mobilephone/logout.asp",helpurl:"mobilephone/cshelp/index.html",adseturl:"mobilephone/systemmanage/setadsearchfields.asp",uploadsmsurl:"mobilephone/uploadsms.asp",uploadcallurl:"mobilephone/uploadcall.asp?mode=mobile",gpssaveurl:"mobilephone/salesmanage/gps/add.asp",uploadgpsmap:"mobilephone/upload.asp?userid={@session}",data:"{\\"dhmanage\\":1,\\"savephone\\":0}"}}}'

In [5]:
t = re.search('(?:session:)(.*)(?=},body)',ses_text).group()[9:-1]
t

'e41a0c259979e294638da27162a3d0e81d2936a2b4485f6d18a926714f1e18b228b250cd00811a94ad5fa748627efba6ebfdc00c5582a0ee'

In [6]:
headers = {"Content-Type":"application/zsml;charset=gb2312"}
data = '{session:"%s",\
        cmdkey:"refresh",\
        datas:[{id:"pagesize",val:"%s"},\
        {id:"dateQD_0",val:"%s"},\
        {id:"dateQD_1",val:"%s"}\
        ]}'%(t,rowsPerPage,start_date,end_date)
# data

In [7]:
%%time
session = requests.Session()
s = session.post("http://192.168.1.30:88/SYSA/mobilephone/salesmanage/contract/billlist.asp",data=data,headers=headers)
org_text = s.text

Wall time: 797 ms


In [8]:
type(org_text)

str

In [9]:
# org_text

In [10]:
%%time
json_obj = demjson.decode(org_text)
# print(json_obj)

Wall time: 110 ms


In [11]:
type(json_obj)

dict

In [12]:
col_dict = json_obj['body']['source']['table']['cols']
# col_dict

In [13]:
col_name = []
for i in col_dict:
    c = i['id']
    col_name.append(c)
# col_name

In [14]:
row_dict = json_obj['body']['source']['table']['rows']
# print(row_dict)

In [15]:
contract_list = pd.DataFrame(row_dict, columns = col_name)
contract_list.shape

(36, 16)

In [16]:
# contract_list.to_csv('contract_list.csv',encoding = 'utf-8',index=False)

In [17]:
# contract_list.head(2)

In [18]:
# contract_list

In [19]:
contract_list.ord = contract_list.ord.astype(int)
contract_list.ord.describe()

count       36.00000
mean     41491.50000
std         18.16197
min      41466.00000
25%      41475.75000
50%      41487.50000
75%      41507.25000
max      41523.00000
Name: ord, dtype: float64

# contract details

In [20]:
# for i in range(contract_list.ord.min(), contract_list.ord.max()):
#     print(i)

In [ ]:
%%time
json_contract_detail_list = []
for i in range(contract_list.ord.min(), contract_list.ord.max()):
    ord_num = i
    headers = {"Content-Type":"application/zsml;charset=gb2312"}
    data = '{session:"%s",datas:[{id:"ord",val:"%s"}]}'%(t,ord_num)
    s = session.post("http://192.168.1.30:88/SYSA/mobilephone/salesmanage/contract/billservice.asp",data=data,headers=headers)
    contract_detail = s.text
    json_contract_detail = demjson.decode(contract_detail)
#     print(json_contract_detail['body']['model'])
    if json_contract_detail['body']['model'] == "message":
        None
    else:
        json_contract_detail_list.append(json_contract_detail)
    # contract_detail

In [ ]:
len(json_contract_detail_list)

In [ ]:
# json_contract_detail

In [ ]:
def make_row(json_object):
    table_dicts = {}
    for item in json_object['body']['bill']['groups']:
        for field in item['fields']:
            if set(("text", "caption")) <= set(field):
                table_dicts[field['caption']]=field['text']
            elif 'text' not in field:
                if 'caption' not in field:
                    None
                else:
                    table_dicts[field['caption']]=None
    return table_dicts

In [ ]:
j_list = []
for json_contract_detail in json_contract_detail_list:
    dict_data = make_row(json_contract_detail)
    columns = list(dict_data.keys())
    values = list(dict_data.values())
    arr_len = len(values)
    df_temp = pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
    j_list.append(df_temp)
# j_list

In [ ]:
len(j_list)

In [ ]:
dfs = pd.concat(j_list,ignore_index = True)
# dfs

In [ ]:
dfs.to_csv('CRM_API_当日合同明细1.csv',encoding = 'gbk')